# Projeto 1

## Tarefa 1

In [1]:
import glob
import cv2
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd


pi = math.pi

In [2]:
#Função para usar o Blob
def blob_detec():
    #Blob
    # Set up the detector with default parameters.
    params = cv2.SimpleBlobDetector_Params()
    # Set blob color (0=black, 255=white)
    params.filterByColor = True
    params.blobColor = 255
    # Filter by Area
    params.filterByArea = False
    # params.minArea = 1
    # #params.maxArea = 20000
    # Filter by Circularity
    params.filterByCircularity = False
    params.minCircularity = 0.9
    #params.maxCircularity = 1
    # Filter by Convexity
    params.filterByConvexity = False
    params.minConvexity = 0.95
    #params.maxConvexity = 0.9
    # Filter by Inertia
    params.filterByInertia = True
    params.minInertiaRatio = 0.97
    #params.maxInertiaRatio = 1
    # Set up the detector with default parameters.
    detector = cv2.SimpleBlobDetector_create(params)
    return detector

def TratamentoImagem(imagem):
    img_hsv = cv2.cvtColor(imagem, cv2.COLOR_BGR2HSV)
    (canal_H, canal_S, canal_V) = cv2.split(img_hsv)
    
    thresh = np.where((((canal_S<160)|(canal_V<80))),255,0).astype(np.uint8)
    
    kernel = np.ones((5,5),np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    #plt.imshow(opening)
    #plt.show()
    
    return opening

#Função para idenficar os contornos da imagem
def IdentificaContorno(imagem):
    contours,hierarchy = cv2.findContours(imagem, 1, 2)
    return contours

#Identifica borda
def IdentificaBorda(imagemtrtd):
    detector = blob_detec()
    KP = detector.detect(imagemtrtd)
    if (len(KP)==1):
        statusBorda = 'ok'
    else:
        statusBorda = 'nok'
    return statusBorda

def IdentificaTamanhos(ctn, hull,imagemtrtd,imgvazia,tamanho_esteira,tamanho_imagem):
    #Área da esteira
    raio_perfeito = int((25*tamanho_imagem)/tamanho_esteira)
    area_perfeita = pi*(raio_perfeito**2)
    
    #Área do contorno
    img = cv2.drawContours(imgvazia,[hull],0,(255,0,0),1)
    area = cv2.contourArea(ctn)
    equi_diameter = np.sqrt(4*area/np.pi)
    
    #Relação das áreas
    relacao_area = area/area_perfeita

    #Encontrando o diâmetro
    (x,y),radius = cv2.minEnclosingCircle(ctn)
    center = (int(x),int(y))
    radius = int(radius)
    diametro = (equi_diameter*tamanho_esteira)/tamanho_imagem
    
    

    #Encontrando a relação A/B dos diâmetro
    (xE, yE), (MA, ma), angle = cv2.fitEllipse(ctn)
    diametros_maior = (MA*esteira_mm)/tamanho_imagem
    diametros_menor = (ma*esteira_mm)/tamanho_imagem
    relacaoAB = diametros_menor/diametros_maior
    
    return relacao_area, diametro, relacaoAB
    
def TesteQualidade(relacao_area,diametro,relacaoAB):
    if (relacao_area>0.98):
        #Teste de diâmetro
        if ((diametro>=49.5) and (diametro<=50.5)):
            diametroArredondado = str(round(diametro, 1))
            statusDiametro = 'ok'
            
            #Teste de relação AB
            if ((relacaoAB>=0.95) and (relacaoAB<=1.05)):
                relacaoABArredondado = str(round(relacaoAB, 3))
                statusRelacaoAB = 'ok'
                statusTamanho = 'ok'
                return statusDiametro, diametroArredondado, statusRelacaoAB, relacaoABArredondado, statusTamanho
            else: 
                statusRelacaoAB = 'ok'
                statusTamanho = 'nok'
                return statusDiametro, diametroArredondado, statusRelacaoAB, relacaoABArredondado, statusTamanho
        else:
            diametroArredondado = str(round(diametro, 1))
            statusDiametro = 'nok'
            relacaoABArredondado = str(round(relacaoAB, 3))
            if ((relacaoAB>=0.95) and (relacaoAB<=1.05)):
                statusRelacaoAB = 'ok'
                statusTamanho = 'nok'
                return statusDiametro, diametroArredondado, statusRelacaoAB, relacaoABArredondado, statusTamanho
            else: 
                statusRelacaoAB = 'ok'
                statusTamanho = 'nok'
                return statusDiametro, diametroArredondado, statusRelacaoAB, relacaoABArredondado, statusTamanho
        

def testandoTudo(images):
    """Tratamento de imagem"""
    img_tratada = TratamentoImagem(images)
    #plt.imshow(opening)
    #plt.show()
    
    """Encontrando os contornos"""
    contornos = IdentificaContorno(img_tratada)
    
    (h,w) = img_tratada.shape
    fig_pos1 = np.zeros((h,w), dtype='uint8')
    
    fig_contor = images
    
    """Realizando testes"""
    for c in contornos:
        hull = cv2.convexHull(c)
        area = cv2.contourArea(c)
        if (area> 99000):
            
            img = cv2.drawContours(img_tratada,[hull],0,(255,255,255),thickness=cv2.FILLED)
            
            fig_contor = img
            
            #Teste de borda
            situ_borda = IdentificaBorda(img_tratada)
            
            if situ_borda=='ok':
                img = cv2.drawContours(fig_pos1,[hull],0,(255,0,0),1)
                area = cv2.contourArea(c)
                areas[i] = area
                equi_diameter = np.sqrt(4*area/np.pi)
                rel_area = area/area_perfeita

                #Encontrando o diâmetro
                (x,y),radius = cv2.minEnclosingCircle(c)
                center = (int(x),int(y))
                radius = int(radius)
                diametros = round(((equi_diameter*esteira_mm)/w),2)

                #Encontrando a relação A/B
                (xE, yE), (MA, ma), angle = cv2.fitEllipse(c)
                diametros_maior = (MA*esteira_mm)/w
                diametros_menor = (ma*esteira_mm)/w
                relacao = round((diametros_menor/diametros_maior),3)
                
                if ((diametros>=49.5) and (diametros<=50.5) and (rel_area>0.98)):
                    diametro_ok = 'ok'

                    if ((relacao>=0.95) and (relacao<=1.05)):
                        relacao_ok = 'ok'
                        tamanho_ok = 'ok'
                        return situ_borda, diametros, diametro_ok, relacao, relacao_ok, tamanho_ok
                    else: 
                        relacao_ok = 'nok'
                        tamanho_ok = 'nok'
                        return situ_borda, diametros, diametro_ok, relacao, relacao_ok, tamanho_ok

                else: 
                    diametro_ok = 'nok'
                    tamanho_ok = 'nok'
                    return situ_borda, diametros, diametro_ok, '-', '-', tamanho_ok
                        
                
            else:
                situ_borda = 'nok'
                return situ_borda,'-','-','-','-','-'
        else:
            situ_borda = 'nok'
            return situ_borda,'-','-','-','-','-'

# Implementado

In [ ]:
import glob
import cv2
import matplotlib.pyplot as plt

#path="NOK_borda\*.jpg"
#path="NOK_tamanho_forma\*.jpg"
#path="OK\*.jpg" #Replace with your folder 
path="ALL\*.jpg" 
k=glob.glob(path)

images=[cv2.imread(images) for images in glob.glob(path)]

print(len(images))     #number of images in folders

#plt.figure(figsize=(200, 800)) 

esteira_mm = 65

areas = [0]*len(images)
diametros = [0]*len(images)
diametros_maiores = [0]*len(images)
diametros_menores = [0]*len(images)
relacao = [0]*len(images)

raio_perfeito = int((25*508)/65)
area_perfeita = pi*(raio_perfeito**2)

borda_ok = ['nok']*len(images)
tamanho_ok = ['-']*len(images)
diametro_ok = ['-']*len(images)
relacao_ok = ['-']*len(images)

for i in range(len(images)):
    
    """Tratamento de imagem"""
    img_tratada = TratamentoImagem(images[i])
    #plt.imshow(opening)
    #plt.show()
    
    """Encontrando os contornos"""
    contornos = IdentificaContorno(img_tratada)
    
    (h,w) = img_tratada.shape
    fig_pos1 = np.zeros((h,w), dtype='uint8')
    
    fig_contor = images[i]
    
    """Realizando testes"""
    for c in contornos:
        hull = cv2.convexHull(c)
        area = cv2.contourArea(c)
        if (area> 99000):
            
            img = cv2.drawContours(img_tratada,[hull],0,(255,255,255),thickness=cv2.FILLED)
            
            fig_contor = img
            
            #Teste de borda
            situ_borda = IdentificaBorda(img_tratada)
            borda_ok[i] = situ_borda
            
            if situ_borda=='ok':
                img = cv2.drawContours(fig_pos1,[hull],0,(255,0,0),1)
                area = cv2.contourArea(c)
                areas[i] = area
                equi_diameter = np.sqrt(4*area/np.pi)
                rel_area = area/area_perfeita

                #Encontrando o diâmetro
                (x,y),radius = cv2.minEnclosingCircle(c)
                center = (int(x),int(y))
                radius = int(radius)
                diametros[i] = round(((equi_diameter*esteira_mm)/w),2)

                #Encontrando a relação A/B
                (xE, yE), (MA, ma), angle = cv2.fitEllipse(c)
                diametros_maiores[i] = (MA*esteira_mm)/w
                diametros_menores[i] = (ma*esteira_mm)/w
                relacao[i] = round((diametros_menores[i]/diametros_maiores[i]),3)
                
                if ((diametros[i]>=49.5) and (diametros[i]<=50.5) and (rel_area>0.98)):
                    diametro_ok[i] = 'ok'

                    if ((relacao[i]>=0.95) and (relacao[i]<=1.05)):
                        relacao_ok[i] = 'ok'
                        tamanho_ok[i] = 'ok'
                    else: 
                        relacao_ok[i] = 'nok'
                        tamanho_ok[i] = 'nok'

                else: 
                    diametro_ok[i] = 'nok'
                    tamanho_ok[i] = 'nok'
                        
                
            else:
                borda_ok[i] = 'nok'
                
            
    #plt.subplot(1,len(images),i+1)
    #plt.imshow(fig_pos1,cmap='gray')

    
#plt.show()
    
print("Check bordas:", borda_ok)    
print("Check diametros:", diametro_ok)
print("Diametros:", diametros)
print("Check relacao:", relacao_ok)
print("Relação AB:", relacao)
print("Check bordas:", borda_ok) 
print("Check tamanho:", tamanho_ok)

In [6]:
import glob
import cv2
import matplotlib.pyplot as plt

#path="NOK_borda\*.jpg"
#path="NOK_tamanho_forma\*.jpg"
#path="OK\*.jpg" #Replace with your folder 
path="ALL\*.jpg" 
k=glob.glob(path)

images=[cv2.imread(images) for images in glob.glob(path)]

print(len(images))     #number of images in folders

#plt.figure(figsize=(200, 800)) 

esteira_mm = 65

areas = [0]*len(images)
diametros = [0]*len(images)
diametros_maiores = [0]*len(images)
diametros_menores = [0]*len(images)
relacao = [0]*len(images)

raio_perfeito = int((25*508)/65)
area_perfeita = pi*(raio_perfeito**2)

borda_ok = ['nok']*len(images)
tamanho_ok = ['-']*len(images)
diametro_ok = ['-']*len(images)
relacao_ok = ['-']*len(images)

for i in range(len(images)):
    
    a,b,c,d,e,f = testandoTudo(images[i])
    borda_ok[i] = a
    diametros[i] = b
    diametro_ok[i] = c
    relacao[i] = d
    relacao_ok[i] = e
    tamanho_ok[i] = f
                
    #plt.subplot(1,len(images),i+1)
    #plt.imshow(fig_pos1,cmap='gray')

    
#plt.show()
    
print("Check bordas:", borda_ok)    
print("Check diametros:", diametro_ok)
print("Diametros:", diametros)
print("Check relacao:", relacao_ok)
print("Relação AB:", relacao)
print("Check bordas:", borda_ok) 
print("Check tamanho:", tamanho_ok)

30
Check bordas: ['nok', 'ok', 'nok', 'ok', 'nok', 'ok', 'nok', 'nok', 'ok', 'ok', 'nok', 'nok', 'nok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok']
Check diametros: ['-', 'nok', '-', 'nok', '-', 'ok', '-', '-', 'nok', 'nok', '-', '-', '-', 'nok', 'nok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok']
Diametros: ['-', 48.1, '-', 45.53, '-', 49.66, '-', '-', 48.47, 49.34, '-', '-', '-', 49.47, 49.37, 49.75, 49.66, 49.74, 49.72, 49.79, 49.64, 49.54, 49.64, 49.59, 49.94, 49.81, 49.69, 49.66, 49.58, 49.81]
Check relacao: ['-', '-', '-', '-', '-', 'ok', '-', '-', '-', '-', '-', '-', '-', '-', '-', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok', 'ok']
Relação AB: ['-', '-', '-', '-', '-', 1.007, '-', '-', '-', '-', '-', '-', '-', '-', '-', 1.004, 1.007, 1.009, 1.006, 1.009, 1.004, 1.002, 1.007, 1.007, 1.005, 1.006, 1.003, 1.006, 1.005, 1.001]
Check bordas: ['

In [7]:
#tabela = [k, borda_ok, diametros, diametro_ok, relacao, relacao_ok,tamanho_ok]
#print(tabela)

data = {'Arquivo':  k,
        'Teste borda': borda_ok,
        'Diametro': diametros,
        'Teste diametro': diametro_ok,
        'Relação AB': relacao,
        'Teste relação AB': relacao_ok,
        'Teste tamanho': tamanho_ok
        }

df = pd.DataFrame(data)
df


,Arquivo,Teste borda,Diametro,Teste diametro,Relação AB,Teste relação AB,Teste tamanho
0,ALL\Fig_NOK_01.jpg,nok,-,-,-,-,-
1,ALL\Fig_NOK_02.jpg,ok,48.1,nok,-,-,nok
2,ALL\Fig_NOK_03.jpg,nok,-,-,-,-,-
3,ALL\Fig_NOK_04.jpg,ok,45.53,nok,-,-,nok
4,ALL\Fig_NOK_05.jpg,nok,-,-,-,-,-
5,ALL\Fig_NOK_06.jpg,ok,49.66,ok,1.007,ok,ok
6,ALL\Fig_NOK_07.jpg,nok,-,-,-,-,-
7,ALL\Fig_NOK_08.jpg,nok,-,-,-,-,-
8,ALL\Fig_NOK_09.jpg,ok,48.47,nok,-,-,nok
9,ALL\Fig_NOK_10.jpg,ok,49.34,nok,-,-,nok


In [5]:
#Criando um TXT com os dados
with open('resultado.txt', 'a') as f:
    dfAsString = df.to_string(header=False, index=False)
    f.write(dfAsString)

# Original

In [ ]:
import glob
import cv2
import matplotlib.pyplot as plt

#path="NOK_borda\*.jpg"
path="NOK_tamanho_forma\*.jpg"
#path="OK\*.jpg" #Replace with your folder 
#path="ALL\*.jpg" 
k=glob.glob(path)
images=[cv2.imread(images) for images in glob.glob(path)]

print(len(images))     #number of images in folders

plt.figure(figsize=(200, 800)) 

esteira_mm = 65

areas = [0]*len(images)
diametros = [0]*len(images)
diametros_maiores = [0]*len(images)
diametros_menores = [0]*len(images)
relacao = [0]*len(images)

raio_perfeito = int((25*508)/65)
area_perfeita = pi*(raio_perfeito**2)

borda_ok = ['nok']*len(images)
tamanho_ok = ['-']*len(images)
diametro_ok = ['-']*len(images)
diametro_status = ['-']*len(images)
relacao_ok = ['-']*len(images)
relacao_status = ['-']*len(images)

for i in range(len(images)):
    #plt.imshow(images[i])
    #plt.show()
    #cv2.imshow("images",images[i])
    #cv2.waitKey(0)
    
    """Tratamento de imagem"""
    img_hsv = cv2.cvtColor(images[i], cv2.COLOR_BGR2HSV)
    (canal_H, canal_S, canal_V) = cv2.split(img_hsv)
    
    thresh = np.where((((canal_S<160)|(canal_V<80))),255,0).astype(np.uint8)
    
    kernel = np.ones((5,5),np.uint8)
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)
    #plt.imshow(opening)
    #plt.show()
    
    """Encontrando os contornos"""
    contours,hierarchy = cv2.findContours(opening, 1, 2)
    #print("\nNumber of objects detected:", len(contours))
    
    (h,w) = opening.shape
    fig_pos1 = np.zeros((h,w), dtype='uint8')
    

    fig_contor = images[i]

    for c in contours:
        hull = cv2.convexHull(c)
        area = cv2.contourArea(c)
        #img = cv2.drawContours(thresh,[hull],0,(156,156,156),1)
        #print("\nArea:",area)
        #if (area> 116800):
        if (area> 90000):
            
            img = cv2.drawContours(opening,[hull],0,(255,255,255),thickness=cv2.FILLED)
            
            fig_contor = img
        
            detector = blob_detec()
            KP = detector.detect(fig_contor)
            #print("Nro de blobs: ",len(KP))
            
            
            
            if (len(KP)==1):
                borda_ok[i] = 'ok'
                
                img = cv2.drawContours(fig_pos1,[hull],0,(255,0,0),1)
                area = cv2.contourArea(c)
                #print("\nArea:",area)
                areas[i] = area
                #print("Relação de areas:",area/area_perfeita)
                equi_diameter = np.sqrt(4*area/np.pi)
                rel_area = area/area_perfeita

                #Encontrando o diâmetro
                (x,y),radius = cv2.minEnclosingCircle(c)
                center = (int(x),int(y))
                radius = int(radius)
                diametros[i] = (equi_diameter*esteira_mm)/508



                #Encontrando a relação A/B
                (xE, yE), (MA, ma), angle = cv2.fitEllipse(c)
                diametros_maiores[i] = (MA*esteira_mm)/508
                diametros_menores[i] = (ma*esteira_mm)/508
                relacao[i] = diametros_menores[i]/diametros_maiores[i]
                
                if (rel_area>0.98):
                    if ((diametros[i]>=49.5) and (diametros[i]<=50.5)):
                        diametro_ok[i] = str(round(diametros[i], 1))
                        diametro_status[i] = 'ok'
                        
                        if ((relacao[i]>=0.95) and (relacao[i]<=1.05)):
                            relacao_ok[i] = str(round(relacao[i], 3))
                            relacao_status[i] = 'ok'
                            tamanho_ok[i] = 'ok'
                        else: 
                            tamanho_ok[i] = 'nok'
                            relacao_status[i] = 'nok'
                        
    
                    else: 
                        tamanho_ok[i] = 'nok'
                        diametro_status[i] = 'ok'
                        
                else:
                    tamanho_ok[i] = 'nok'
            
                
            else:
                borda_ok[i] = 'nok'
        
            
            
            
            
    plt.subplot(1,len(images),i+1)
    plt.imshow(fig_pos1,cmap='gray')

    
plt.show()
    
print("Check bordas:", borda_ok)    
print("Valor diametros:", diametro_ok)
print("Check diametros:", diametro_status)
print("Valor relacao:", relacao_ok)
print("Check relacao:", relacao_status)
print("Check tamanho:", tamanho_ok)
print("\nÁreas") 
print("Média:",np.mean(areas))
print("Máx",np.max(areas))
print("Min:", np.min(areas))
print("Diametros")
print("Média:",np.mean(diametros))
print("Máx",np.max(diametros))
print("Min:", np.min(diametros))
print("Diametros maiores:")
print("Média:",np.mean(diametros_maiores))
print("Máx",np.max(diametros_maiores))
print("Min:", np.min(diametros_maiores))
print("Diametros menores:")
print("Média:",np.mean(diametros_menores))
print("Máx",np.max(diametros_menores))
print("Min:", np.min(diametros_menores))
print("Relação A/B:", relacao)


# TESTE COM AS FUNÇÃO (deu ruim :( )

In [ ]:
import glob
import cv2
import matplotlib.pyplot as plt

#path="NOK_borda\*.jpg"
#path="NOK_tamanho_forma\*.jpg"
#path="OK\*.jpg" #Replace with your folder 
path="ALL\*.jpg" 
k=glob.glob(path)
images=[cv2.imread(images) for images in glob.glob(path)]

print(len(images))     #number of images in folders

plt.figure(figsize=(200, 800)) 

esteira_mm = 65

areas = [0]*len(images)
diametros = [0]*len(images)
diametros_maiores = [0]*len(images)
diametros_menores = [0]*len(images)
relacao = [0]*len(images)

raio_perfeito = int((25*508)/65)
area_perfeita = pi*(raio_perfeito**2)

borda_ok = ['nok']*len(images)
tamanho_ok = ['-']*len(images)
diametro_ok = ['-']*len(images)
relacao_ok = ['-']*len(images)

for i in range(len(images)):
    #plt.imshow(images[i])
    #plt.show()
    #cv2.imshow("images",images[i])
    #cv2.waitKey(0)
    
    """Tratamento de imagem"""
    img_tratada = TratamentoImagem(images[i])
    #plt.imshow(opening)
    #plt.show()
    
    """Encontrando os contornos"""
    contornos = IdentificaContorno(img_tratada)
    
    (h,w) = img_tratada.shape
    fig_pos1 = np.zeros((h,w), dtype='uint8')
    
    fig_contor = images[i]

    for c in contornos:
        hull = cv2.convexHull(c)
        area = cv2.contourArea(c)
        #img = cv2.drawContours(thresh,[hull],0,(156,156,156),1)
        #print("\nArea:",area)
        #if (area> 116800):
        if (area> 99000):
            
            img = cv2.drawContours(img_tratada,[hull],0,(255,255,255),thickness=cv2.FILLED)
            
            fig_contor = img
            
            #Teste de borda
            situ_borda = IdentificaBorda(img_tratada)
            borda_ok[i] = situ_borda
            #rel_area, diametro, relacaoAB = IdentificaTamanhos(c, hull, img_tratada,fig_pos1,esteira_mm,w)
            
            if situ_borda=='ok':
                img = cv2.drawContours(fig_pos1,[hull],0,(255,0,0),1)
                area = cv2.contourArea(c)
                #print("\nArea:",area)
                areas[i] = area
                #print("Relação de areas:",area/area_perfeita)
                equi_diameter = np.sqrt(4*area/np.pi)
                rel_area = area/area_perfeita

                #Encontrando o diâmetro
                (x,y),radius = cv2.minEnclosingCircle(c)
                center = (int(x),int(y))
                radius = int(radius)
                diametros[i] = round(((equi_diameter*esteira_mm)/508),2)



                #Encontrando a relação A/B
                (xE, yE), (MA, ma), angle = cv2.fitEllipse(c)
                diametros_maiores[i] = (MA*esteira_mm)/508
                diametros_menores[i] = (ma*esteira_mm)/508
                relacao[i] = round((diametros_menores[i]/diametros_maiores[i]),3)
                
                if ((diametros[i]>=49.5) and (diametros[i]<=50.5) and (rel_area>0.98)):
                    #diametro_ok[i] = str(round(diametros[i], 1))
                    diametro_ok[i] = 'ok'

                    if ((relacao[i]>=0.95) and (relacao[i]<=1.05)):
                        #relacao_ok[i] = str(round(relacao[i], 3))
                        relacao_ok[i] = 'ok'
                        tamanho_ok[i] = 'ok'
                    else: 
                        relacao_ok[i] = 'nok'
                        tamanho_ok[i] = 'nok'


                else: 
                    diametro_ok[i] = 'nok'
                    tamanho_ok[i] = 'nok'
                        
                
            
                
            else:
                borda_ok[i] = 'nok'
                
                #Teste de tamanhos
                #rel_area, diametro, relacaoAB = IdentificaTamanhos(c, hull, img_tratada,fig_pos1,esteira_mm,h)
                #print("\n",situ_borda) 
                #print(rel_area)
                #print(diametro)
                #print(relacaoAB)
                #statusDiametro, diametroFinal, statusRelacaoAB, relacaoABFinal, statusTamanho  = TesteQualidade(rel_area,diametro,relacaoAB) 
                #print(statusDiametro, diametroFinal, statusRelacaoAB, relacaoABFinal, statusTamanho)
            
    plt.subplot(1,len(images),i+1)
    plt.imshow(fig_pos1,cmap='gray')

    
plt.show()
    
print("Check bordas:", borda_ok)    
print("Check diametros:", diametro_ok)
print("Diametros:", diametros)
print("Check relacao:", relacao_ok)
print("Relação AB:", relacao)
print("Check tamanho:", tamanho_ok)

# Apoios

In [ ]:

#fig = cv2.imread("OK/Fig_OK_15.jpg")
#fig = cv2.imread("NOK_tamanho_forma\Fig_NOK_01.jpg")
fig = cv2.imread("NOK_borda/Fig_NOK_14.jpg")
#fig = cv2.imread("NOK_superficie/Fig_NOK_Superf_03.jpg")

#plt.imshow(fig)
#plt.show()


(h,w,c) = fig.shape

figRGB = cv2.cvtColor(fig,cv2.COLOR_BGR2RGB)
plt.imshow(figRGB)
plt.show()


img_hsv = cv2.cvtColor(fig, cv2.COLOR_BGR2HSV)
(canal_H, canal_S, canal_V) = cv2.split(img_hsv)
plt.figure("Hue")
plt.subplot(1,3,1)
plt.imshow(canal_H,cmap='gray')
plt.subplot(1,3,2)
plt.imshow(canal_S,cmap='gray')
plt.subplot(1,3,3)
plt.imshow(canal_V,cmap='gray')
plt.show()

#print("H max:",np.max(canal_H))
#print("H min:", np.min(canal_H))
#print("S max:",np.max(canal_S))
#print("S min:", np.min(canal_S))
#print("V max:",np.max(canal_V))
#print("V min:", np.min(canal_V))

#print(canal_H)


#Separando as matrizes
#[B,G,R] = cv2.split(figRGB)
#plt.imshow(G, cmap='gray')
#plt.show()

#Binarizando a imagem para fazer o cálculo
#fig_bin = np.where((B>120), 255, 0).astype(np.uint8)
#plt.imshow(fig_bin, cmap='gray')
#plt.show()

#gamma = adjust_gamma(canal_S, 5)
#plt.imshow(gamma,cmap='gray')
#plt.show()
#(canal_H>101)&(canal_V<170)
#&(canal_V<200) & (canal_S<130)
thresh = np.where((((canal_S<160))),255,0).astype(np.uint8)
plt.imshow(thresh,cmap='gray')
plt.show()


dst = cv2.GaussianBlur(thresh,(3,3),cv2.BORDER_DEFAULT)
plt.imshow(dst,cmap='gray')
plt.show()

#circles = cv2.HoughCircles(fig, cv2.HOUGH_GRADIENT,1,1)
#print(circles)

kernel = np.ones((9,9),np.uint8)
opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel)

contours,hierarchy = cv2.findContours(opening, 1, 2)
print("Number of objects detected:", len(contours))

#contours[-5] possui o contorno do circulo na figura 1

fig_pos1 = np.zeros((h,w), dtype='uint8')

fig_contor = fig
raio_perfeito = int((25*508)/65)
area_perfeita = pi*(raio_perfeito**2)

for c in contours:
    hull = cv2.convexHull(c)
    #img = cv2.drawContours(fig,[hull],0,(255,255,0),2)
    area = cv2.contourArea(c)
    #print("\nArea:",area)
    
    #print("\nArea:",area)
    if (area>99000):
        #img = cv2.drawContours(fig_pos1,[hull],0,(255,255,255),thickness=1)
        #equi_diameter = np.sqrt(4*area/np.pi)
        #print("Diameter:", equi_diameter)
        #(x,y),radius = cv2.minEnclosingCircle(c)
        #center = (int(x),int(y))
        #radius = int(radius)
        cv2.circle(fig_pos1,center,raio_perfeito,(156,156,156),1)
        
        #fig_contor = cv2.drawContours(img,[hull],0,(156,156,156), 2)
        img = cv2.drawContours(fig_pos1,[hull],0,(255,255,255),1)
        area = cv2.contourArea(c)
        #print("\nArea:",area)
        areas[i] = area
        print("Relação de areas:",area/area_perfeita)

        #Encontrando o diâmetro
        (x,y),radius = cv2.minEnclosingCircle(c)
        center = (int(x),int(y))
        radius = int(radius*2)
        diametros[i] = (radius*esteira_mm)/508

        #Encontrando a relação A/B
        (xE, yE), (MA, ma), angle = cv2.fitEllipse(c)
        diametros_maiores[i] = (MA*esteira_mm)/508
        diametros_menores[i] = (ma*esteira_mm)/508
        relacao[i] = diametros_menores[i]/diametros_maiores[i]
        #print("MA:", MA)
        #print("ma:", ma)
        
        fig_contor = img
        

print("Teste")
plt.imshow(fig_contor, cmap='gray')
plt.show()

#ret2, fig_contor2 = cv2.threshold(fig_contor,250,255,0)
#plt.imshow(fig_contor2, cmap='gray')
#plt.show()

#fig_contor3 = cv2.cvtColor(fig_contor,cv2.COLOR_BGR2RGB)
#plt.imshow(fig_contor3)
#plt.show()


#detector = blob_detec()

# Detect blobs
#KP = detector.detect(fig_contor)
#print("Nro de blobs: ",len(KP))
# List parameters (X,Y,size,ang) of each detected keypoints
#img1_text = fig_contor
#i=1
#for KPi in KP:
#    print("Blob_", i, ": X= ", KPi.pt[0], " Y= ", KPi.pt[1], " size=", KPi.size**2, " ang=", KPi.angle)
#    img1_text = cv2.putText(img1_text, str(i), (int(KPi.pt[0]),int(KPi.pt[1])), cv2.FONT_HERSHEY_PLAIN, 1,(255,0,0))
#    i=i+1

#plt.imshow(img1_text, cmap='gray')
#plt.show()



#Green para borda
#HUE e SAT para a superfície


#Canal v - rugosidade
#Canal s - borda

## Loop para rodar a pasta


In [ ]:
import glob
import cv2
import matplotlib.pyplot as plt


path="ALL\*.jpg"    #Replace with your folder 
#path("OK\*.jpg") 
k=glob.glob(path)
images=[cv2.imread(images) for images in glob.glob(path)]

print(len(images))     #number of images in folder
for i in range(len(images)):
    plt.imshow(images[i])
    plt.show()
    #cv2.imshow("images",images[i])
    #cv2.waitKey(0)